In [1]:
import torch
from transformers import pipeline
from datasets import load_dataset
from IPython.display import Audio

In [2]:
from transformers import AutoProcessor, SeamlessM4Tv2Model

C:\Users\Otniel\.conda\envs\p310\lib\site-packages\transformers\deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [3]:
processor = AutoProcessor.from_pretrained("facebook/seamless-m4t-v2-large", torch_dtype=torch.float16, device_map="cuda")
model = SeamlessM4Tv2Model.from_pretrained("facebook/seamless-m4t-v2-large", torch_dtype=torch.float16, device_map="cuda").eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
dataset = load_dataset("arabic_speech_corpus", split="test", streaming=True)
audio_sample = next(iter(dataset))["audio"]

In [5]:
audio_inputs = processor(audios=audio_sample["array"], sampling_rate=16000, return_tensors="pt", device='cuda')

In [6]:
text_inputs = processor(text = "Hello, my dog is cute", src_lang="eng", return_tensors="pt")

In [9]:
audio_array_from_text = model.generate(input_ids=text_inputs['input_ids'].to('cuda'), attention_mask=text_inputs['attention_mask'].to('cuda'), 
                                       tgt_lang="rus", text_num_beams=4, text_do_sample=True)[0].cpu().numpy().squeeze()
audio_array_from_audio = model.generate(input_features=audio_inputs['input_features'].to('cuda'), attention_mask=audio_inputs['attention_mask'].to('cuda'), 
                                        tgt_lang="rus", text_num_beams=4, text_do_sample=True)[0].cpu().numpy().squeeze()

In [10]:
Audio(audio_array_from_text, rate=16000)

In [14]:
Audio(audio_sample["array"], rate=48000)

In [19]:
Audio(audio_array_from_audio, rate=16000)

In [19]:
audio_inputs

{'input_features': tensor([[[-5.4686, -5.3245, -5.1805,  ..., -6.0474, -6.0581, -6.0663],
         [-5.4686, -5.3245, -5.1805,  ..., -6.0474, -6.0581, -6.0663],
         [-5.4686, -5.3245, -5.1805,  ..., -6.0474, -6.0581, -6.0663],
         ...,
         [-5.4686, -5.3245, -5.1805,  ..., -6.0474, -6.0581, -6.0663],
         [-5.4686, -5.3245, -5.1805,  ..., -6.0474, -6.0581, -6.0663],
         [-5.4686, -5.3245, -5.1805,  ..., -6.0474, -6.0581, -6.0663]]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]], dtype=torch.int32)}